In [26]:
import pandas as pd

#need to cross validate...

moviesFull1 = pd.read_csv('dataset/movie_dataset.csv')
moviesFull2 = pd.read_csv('newdata/movies_metadata.csv')
keywords = pd.read_csv('newdata/keywords.csv')

print(moviesFull1.loc[0][['keywords','cast','genres','director']])

print(moviesFull2.loc[moviesFull2["title"] == "avatar"][['tagline','genres']])



f = open("text.txt", "a")
f.write(moviesFull1.loc[0]['keywords']+"\n")
f.write(moviesFull1.loc[0]['cast']+"\n")
f.write(moviesFull1.loc[0]['genres']+"\n")
f.write(moviesFull1.loc[0]['director']+"\n")

print(moviesFull2.loc[moviesFull2["title"] == "Avatar"]['tagline']+"\n")
print(moviesFull2.loc[moviesFull2["title"] == "Avatar"]['genres']+"\n")

print(keywords.loc[keywords["id"] == moviesFull2.loc[moviesFull2["title"] == "Avatar"]["id"]])



f.close()



keywords    culture clash future space war space colony so...
cast        Sam Worthington Zoe Saldana Sigourney Weaver S...
genres               Action Adventure Fantasy Science Fiction
director                                        James Cameron
Name: 0, dtype: object
Empty DataFrame
Columns: [tagline, genres]
Index: []
14551    Enter the World of Pandora.\n
Name: tagline, dtype: object
14551    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
Name: genres, dtype: object


C:\Users\jackson\AppData\Local\Temp\ipykernel_24796\2909130930.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  moviesFull2 = pd.read_csv('newdata/movies_metadata.csv')


ValueError: Can only compare identically-labeled Series objects